In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
base_model = "mistralai/Mistral-7B-v0.1" #bn22/Mistral-7B-Instruct-v0.1-sharded
dataset_name= "riskyhomo/ayn_rand"
new_model="riskyhomo/Ayn_Rand_BB"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset= load_dataset(dataset_name, split="train")
dataset["text"][0]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model)


In [ ]:
tokenizer= AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_size="right"
tokenizer.add_oes_token =True

tokenizer.add_bos_token , tokenizer.add_oes_token

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(lora_alpha=16,
                        lora_dropout=0.1,
                        r=64,
                        bias="none",
                        task_type="CASUAL_LM",
                        target_modules=["q_proj", "v_proj"])
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)
model.config.use_cache=True

In [ ]:
trainer.model.push_to_hub(new_model, use_temp_dir=True)

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt="What is love?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result=pipe(f"{prompt}")
print(result[0]["generated_text"])